In [3]:
# loading GWSNR class from the gwsnr package
from gwsnr import GWSNR
import numpy as np

# initializing the GWSNR class with default parameters
gwsnr = GWSNR(snr_type='interpolation_aligned_spins_jax', npool=8, gwsnr_verbose=False)

# signal-to-noise ratio for a binary black hole merger with masses 10, 10 Msun at a luminosity distance of 100 Mpc
snrs = gwsnr.snr(mass_1=np.array([30.0]), mass_2=np.array([30.0]), luminosity_distance=np.array([1000.0]), a_1=np.array([0.0]), a_2=np.array([0.0]))

# print the signal-to-noise ratio
print('Computed SNRs:\n', snrs)


Initializing GWSNR class...

psds not given. Choosing bilby's default psds
Interpolator will be loaded for L1 detector from ./interpolator_pickle/L1/partialSNR_dict_0.pickle
Interpolator will be loaded for H1 detector from ./interpolator_pickle/H1/partialSNR_dict_0.pickle
Interpolator will be loaded for V1 detector from ./interpolator_pickle/V1/partialSNR_dict_0.pickle


Computed SNRs:
 {'L1': array([29.31369781]), 'H1': array([18.70657349]), 'V1': array([8.94173241]), 'optimal_snr_net': array([35.90520096])}


In [4]:
# gerneral case, random parameters
# chirp mass can go upto only 95 if f_min=20. to get non zero SNR
np.random.seed(42)
nsamples = 100000
mtot = np.random.uniform(2*4.98, 2*112.5,nsamples)
mass_ratio = np.random.uniform(0.2,1,size=nsamples)
param_dict = dict(
    # convert to component masses
    mass_1 = mtot / (1 + mass_ratio),
    mass_2 = mtot * mass_ratio / (1 + mass_ratio),
    # Fix luminosity distance
    luminosity_distance = 500*np.ones(nsamples),
    # Randomly sample everything else:
    theta_jn = np.random.uniform(0,2*np.pi, size=nsamples),
    ra = np.random.uniform(0,2*np.pi, size=nsamples), 
    dec = np.random.uniform(-np.pi/2,np.pi/2, size=nsamples), 
    psi = np.random.uniform(0,2*np.pi, size=nsamples),
    phase = np.random.uniform(0,2*np.pi, size=nsamples),
    geocent_time = 1246527224.169434*np.ones(nsamples),
    a_1 = np.random.uniform(-0.8,0.8, size=nsamples),
    a_2 = np.random.uniform(-0.8,0.8, size=nsamples),
)

In [5]:
## aligned spins
# numba njit: time=28.8 ms ± 339 μs; jax jit: time= 13.1 ms ± 180 μs (forkserver), time= 13.1 ms ± 141 μs (fork)
%timeit snrs = gwsnr.snr(**param_dict)

144 ms ± 4.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
## no spins
# numba : 64.8 ms
# jax: 17.7 ms

## aligned spins
# numba : 232 ms
# jax: 141 ms

In [9]:
from gwsnr.utils import save_pickle, load_pickle


In [10]:
15*60/89e-3

10112.359550561798

In [ ]:
x10000 improvement in speed